In [ ]:
class Solution(object):
    def __init__(self):
        self.pools = []
    def set_blocks(self, height):
        self.blocks = []
        for i, h in enumerate(height):
            block = {}
            block['index'] = i
            block['height'] = h
            block['left_wall'] = None
            block['right_wall'] = None
            self.blocks.append(block)
    def get_block(self,i):
        for block in self.blocks:
            if block['index'] == i:
                return block
    def set_sorted_heights(self, height):
        # keep ref to orginal indices
        # sort highest to lowest
        sorted_heights = sorted(enumerate(height), key=lambda x: x[1], reverse=True)
        self.sorted_heights = sorted_heights
    def update_sorted_heights(self, left_border, right_border):
        # remove everything between left and right border
        self.sorted_heights = [h for h in self.sorted_heights if not (left_border < h[0] < right_border)]
    def set(self, height):
        self.set_blocks(height)
        self.set_sorted_heights(height)
    def create_pool(self, index, left_wall=None, right_wall=None, relative_depth=None, nr_of_filled_blocks=None, value=None):
        pool = {}
        pool['index'] = index
        pool['left_wall'] = left_wall
        pool['right_wall'] = right_wall
        pool['relative_depth'] = relative_depth
        pool['nr_of_filled_blocks'] = nr_of_filled_blocks          
        pool['contained_indices'] = range(left_wall+1, right_wall) if left_wall is not None and right_wall is not None else []
        pool['value'] = value
        self.pools.append(pool)
        return pool
    def update_pool(self, pool, left_wall=None, right_wall=None, relative_depth=None, nr_of_filled_blocks=None, value=None):
        if left_wall is not None:
            pool['left_wall'] = left_wall
        if right_wall is not None:
            pool['right_wall'] = right_wall
        if relative_depth is not None:
            pool['relative_depth'] = relative_depth
        if nr_of_filled_blocks is not None:
            pool['nr_of_filled_blocks'] = nr_of_filled_blocks
        if value is not None:
            pool['value'] = value

    def create_or_update_pool(self, index, left_wall=None, right_wall=None, relative_depth=None, nr_of_filled_blocks=None, value=None):
        pool = self.get_pool(index)
        if pool:
            self.update_pool(pool, left_wall=left_wall, right_wall=right_wall, relative_depth=relative_depth, nr_of_filled_blocks=nr_of_filled_blocks, value=value)
        else:
            self.create_pool(index, left_wall=left_wall, right_wall=right_wall, relative_depth=relative_depth, nr_of_filled_blocks=nr_of_filled_blocks, value=value)
    def get_pool_as_left_wall(self, left_wall):
        for pool in self.pools:
            if pool['left_wall'] == left_wall:
                return pool
        return None
    def get_pool_as_right_wall(self, right_wall):
        for pool in self.pools:
            if pool['right_wall'] == right_wall:
                return pool
        return None
    def find_left_wall(self, index):
        return self.find_wall(index, moveLeft=True)
    def find_right_wall(self, index):
        return self.find_wall(index, moveLeft=False)
    def find_wall(self,this_index, moveLeft: bool):
        if moveLeft:
            r = range(this_index-1, -1, -1)
        else:
            r = range(this_index+1, len(self.blocks))
        if len(r) == 0:   
            return None
        this_height = self.blocks[this_index]['height']
        sign = -1 if moveLeft else 1
        min_next_height = self.blocks[this_index + sign]['height'] if 0 <= this_index + sign < len(self.blocks) else 0
        if min_next_height >= this_height:
            return None
        return self.find_wall_iteratively(this_height, min_next_height, r)

    def find_wall_iteratively(self,next_height,min_next_height, r=None):
        # go lower by 1 and search, if nothing is found lower by one again untill you hit the next block height. Then exit and return None
        next_height -= 1
        for i in r:
            if self.blocks[i]['height'] > next_height:
                return i
        if next_height == min_next_height:
            return None
        temp = self.find_wall_iteratively(next_height, min_next_height, r)
        return temp
    def create_pools(self):
        i = 0
        # whle i is in range of sorted heights and the height is not 0
        while i < len(self.sorted_heights):
            index, height = self.sorted_heights[i]
            i += 1
            # find if index is already the left or right wall of an existing pool
            index_is_already_a_left_wall = self.get_pool_as_left_wall(index)
            if not index_is_already_a_left_wall:
                right_wall = self.find_right_wall(index)
                if right_wall is not None:
                    self.create_pool(index, left_wall=index, right_wall=right_wall)
                    self.update_sorted_heights(index, right_wall)
            index_is_already_a_right_wall = self.get_pool_as_right_wall(index)
            if not index_is_already_a_right_wall:
                left_wall = self.find_left_wall(index)
                if left_wall is not None:
                    self.create_pool(index, left_wall=left_wall, right_wall=index)
                    self.update_sorted_heights(left_wall, index)
    def calculate_pool(self, pool):
        left_h = pool['left_wall']
        right_h = pool['right_wall']
        relative_depth = pool['relative_depth']
        nr_of_filled_blocks = pool['nr_of_filled_blocks']
        value = (right_h - (left_h + 1)) * relative_depth - nr_of_filled_blocks
        return value
    def calculate_pools(self):
        for pool in self.pools:
            pool['value'] = self.calculate_pool(pool)
    def find_relative_depth_and_nr_of_filled_blocks(self, pool):
        # floor is the lowest block between the left and right wall        
        contained_indices = pool['contained_indices']
        contained_values = [self.blocks[i]['height'] for i in contained_indices]

        floor = min(contained_values) if contained_values else 0
        # ceiling is the lowest wall
        left_wall_h = self.blocks[pool['left_wall']]['height'] 
        right_wall_h = self.blocks[pool['right_wall']]['height']
        ceiling = min(left_wall_h, right_wall_h)
        relative_depth = ceiling - floor
        
        # nr of filled blocks is the amount each block between the left and right wall is higher then the floor
        nr_of_filled_blocks = sum(contained_values) - floor * len(contained_values)
        return relative_depth, nr_of_filled_blocks
    def update_relative_depth_and_nr_of_filled_blocks(self):
        for pool in self.pools:
            relative_depth, nr_of_filled_blocks = self.find_relative_depth_and_nr_of_filled_blocks(pool)
            self.update_pool(pool, relative_depth=relative_depth, nr_of_filled_blocks=nr_of_filled_blocks)
    def trap(self, height):
        self.set(height)
        self.create_pools()
        self.update_relative_depth_and_nr_of_filled_blocks()
        self.calculate_pools()

        return sum(pool['value'] for pool in self.pools)


In [54]:
# test
solution = Solution()
sol = solution.trap([4,2,0,3,2,5])

searching for right wall for index 5
return else_find_wall
no right wall found for index 5
searching for left wall for index 5
return if_block_height_greater_then_next_height 0
left wall for index 5 is 0
creating pool
with index 5 left_wall 0 right_wall 5 relative_depth None nr_of_filled_blocks None value None
searching for left wall for index 0
return else_find_wall
no left wall found for index 0
range(1, 5)
[2, 0, 3, 2]


In [55]:
# show sol
for pool in solution.pools:
    print(pool)


{'index': 5, 'left_wall': 0, 'right_wall': 5, 'relative_depth': 4, 'nr_of_filled_blocks': 7, 'contained_indices': range(1, 5), 'value': 9}


In [56]:
for i in range(7,3,-1):  
    print(i)


7
6
5
4
